In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import os

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/media/workstation/Storage/GoogleProject/DeepLearningAlphaC.txt', scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open("TASK")
worksheet_list = spreadsheet.worksheets()
Accepted = spreadsheet.worksheet("Accepted").get_all_records()

In [2]:
# df = pd.DataFrame(Accepted).astype(str)

In [3]:
# StratCode = str(1190)
# u1 = df["Strategy"] == StratCode

# StratName = df.loc[u1].loc[0,"StratName"].replace("SQL", "LIVE")
# TimeFrame = df.loc[u1].loc[0,"TimeFrame"]
# asset_b = df.loc[u1].loc[0, "Asset_B"]
# asset_s = df.loc[u1].loc[0, "Asset_S"]
# if len(asset_b) >= len(asset_s):
#     RefInstrument = asset_b
# else:
#     RefInstrument = asset_s
# ConsecutiveSignalBars = df.loc[u1].loc[0, "HoldingPeriod"]
# OrderType = df.loc[u1].loc[0, "OrderType"]
# NumInstrument = len(RefInstrument.split(","))


In [4]:
def generate_cs(StratName, TimeFrame, StratCode, RefInstrument, ConsecutiveSignalBars, OrderType, NumInstrument, output_folder):
    if TimeFrame == "M_60":
        fo = open("LIVETemplate_DeepLearning_Rotation_M60_R_LIVE.cs", "r+")
        line = fo.readlines()
        print("Using M60 Template!")
    elif TimeFrame in ["M_5", "M_15", "M_30"]:
        fo = open("LIVETemplate_DeepLearning_Rotation_M30ETC_R_LIVE.cs", "r+")
        line = fo.readlines()
        print("Using M30 Template!")

    for i, z in enumerate(line):
        if z.find("LIVETemplate_DeepLearning_Rotation_M30ETC_R_LIVE") != -1:
            line[i] = z.replace("LIVETemplate_DeepLearning_Rotation_M30ETC_R_LIVE", StratName)

        if z.find("LIVETemplate_DeepLearning_Rotation_M60_R_LIVE") != -1:
            line[i] = z.replace("LIVETemplate_DeepLearning_Rotation_M60_R_LIVE", StratName)

            
        if z.find("chg_strategycode") != -1:
            line[i] = z.replace("chg_strategycode", StratCode)

        if z.find("chg_refinstrument") != -1:
            line[i] = z.replace("chg_refinstrument", RefInstrument)
        
        if z.find("chg_consecutive") != -1:
            line[i] = z.replace("chg_consecutive", ConsecutiveSignalBars)
            
        if z.find("chg_order") != -1:
            line[i] = z.replace("chg_order", OrderType)

        if z.find("chg_numinstrument") != -1:
            line[i] = z.replace("chg_numinstrument", NumInstrument)
    
    dir_folder = output_folder + "/{}.cs".format(StratName)
    f = open(dir_folder, 'w')
    
    #f = open(r'Output_Live/{}.cs'.format(StratName), 'w')
    f.writelines(line)
    f.close()
    
def get_code(code):
    if code.find("_") != -1:
        return code.split("_")[1].lower()
    else:
        return code.lower()

In [5]:

output_folder = "20190218_Output_LIVE_ALL"
os.mkdir(output_folder)

In [6]:
for d in Accepted:
    if d["Status_LIVE"] != "d":
        StratCode = str(d["Strategy"])
        StratName = str(d["StratName"]).replace("SQL", "LIVE")
        TimeFrame = str(d["TimeFrame"])
        asset_b = d["Asset_B"]
        asset_s = d["Asset_S"]
        if len(asset_b) >= len(asset_s):
            RefInstrument = asset_b
        else:
            RefInstrument = asset_s
        RefInstrument = RefInstrument.replace(" ", "")
        ConsecutiveSignalBars = str(d["HoldingPeriod"])
        OrderType = str(d["OrderType"])
        NumInstrument = str(len(RefInstrument.split(",")))

        print("=============")
        print(StratName, TimeFrame,ConsecutiveSignalBars, OrderType)
        print(RefInstrument)
        generate_cs(StratName, TimeFrame, StratCode, RefInstrument, ConsecutiveSignalBars, OrderType, NumInstrument, output_folder)
        print("Done")
        print("=============")

_1286_RNNBreakoutW006_R_LIVE M_60 8 0
VWO,XBI,XLK
Using M60 Template!
Done
_1287_RNNBreakoutW012_R_LIVE M_60 8 2
GBPUSD,USDJPY
Using M60 Template!
Done
_1288_RNNBreakoutX004_R_LIVE M_30 8 0
TLT,USDCAD,XOP
Using M30 Template!
Done
_1289_RNNBreakoutY002_R_LIVE M_15 9 0
TBT,XAGUSD
Using M30 Template!
Done
_1290_RNNBreakoutY007_R_LIVE M_15 8 0
LQD,USDJPY
Using M30 Template!
Done
_1300_RNNBreakoutZ001_R_LIVE M_5 7 0
XAGUSD
Using M30 Template!
Done
_1301_RNNBreakoutZ007_R_LIVE M_5 9 0
EURJPY,NZDUSD,XAGUSD
Using M30 Template!
Done


In [ ]:
# for d in Accepted:
#     StratName = d["StratName"]
#     TimeFrame = d["TimeFrame"]
#     RefInstrument = d["Asset_B"] if d["Asset_B"] != "-" else d["Asset_S"]
#     dbTableLong = get_code(d["Code"]) + "b"
#     dbTableShort = get_code(d["Code"]) + "s"
#     ConsecutiveSignalBars = str(d["HoldingPeriod"])
#     OrderType = str(d["OrderType"])
#     print(StratName, TimeFrame, dbTableLong, dbTableShort, ConsecutiveSignalBars, OrderType)
#     print(RefInstrument)
#     print("=============")
#     generate_cs(StratName, TimeFrame, RefInstrument, dbTableLong, dbTableShort, ConsecutiveSignalBars, OrderType)
